# Masked Languaje Model con el sistema RoBERTa, a partir de un corpus de comentarios de redes sociales en español de Costa Rica.

Andrés Fallas

Dayana Marín

Jennifer Villalobos

##Se instalan las librerías necesarias: 

Transformers para el procesamiento del lenguaje natural.

Tensorflow se desinstala, ya que posteriormente se utilizará PyTorch.

In [1]:
!pip uninstall -y tensorflow
!pip install transformers==2.8.0

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Successfully uninstalled tensorflow-2.7.0
     |████████████████████████████████| 563 kB 4.2 MB/s 
     |████████████████████████████████| 131 kB 68.6 MB/s 
     |████████████████████████████████| 895 kB 48.8 MB/s 
     |████████████████████████████████| 5.6 MB 28.1 MB/s 
     |████████████████████████████████| 1.2 MB 36.6 MB/s 
     |████████████████████████████████| 79 kB 6.4 MB/s 
     |████████████████████████████████| 8.4 MB 38.4 MB/s 
     |████████████████████████████████| 138 kB 49.1 MB/s 
     |████████████████████████████████| 127 kB 55.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.6 requires tenso

##Datos
El set de datos corresponde a comentarios extraídos de redes sociales en español de Costa Rica. En esta sección se crea la carpeta "data", donde se almacenarán. Luego de ejecutar este comando, ingrese a la carpeta "data" en la sección izquierda de su patalla. Diríjase a los tres puntos al lado de esta y elija la opción "subir". Por último, seleccione el archivo llamado "dataset". Ahora sí, estamos listos.

In [2]:
import os
#!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILEID' -O dataset.txt
!mkdir data
#!mv dataset.txt data

Por medio de estos comandos se observa el número total de líneas del set se datos, así como unas cuantas líneas del mismo.

In [4]:
# Total de lineas
!wc -l /content/data/dataset.txt

# Algunas de ellas
!shuf -n 5 /content/data/dataset.txt

0 /content/data/dataset.txt
All igual que usted nunca debio generalizar pues muchos costarricenses la ayudaron lo recuerdo bien de cuando yo era una niña cuando esta sra andaba alcoholizada Remordimiento por lo mal que hablo de su pais Tanta trama por algo que ya no significa nada mas basura Que pateticoHe visto Coreanos Rusos y muchisimos pero muchisimos europeos que comprenden lo que ella era Una que descansa en el cielo estrellado junto a Edith Piaf una que llego donde ni sus hijos se va a acercar Si si caguense en mi por decir lo que es cierto la verdad cuando este pais odia a alguien es porque algo bueno habra hecho Bueno mucho artistas son bohemios  y si se tratara de tratar mal a la gente que anda alcoholizada en nuestro pais habria que tratarse mal al 09 de la poblacion Me extraña ese comentario viniendo de un familiar del talentoso Paco Navarrette quien estoy seguro supo apreciar el talento de esa señora Playas del Coco contaminado y veien a tirar mas basura No gracias Que dej

Para el modelo, se toman las primeras 500 líneas  como entranamiento. Y otras 300 para validación.

In [5]:
# Sub-set de entrenamiento
TRAIN_SIZE = 500 #@param {type:"integer"}
!(head -n $TRAIN_SIZE /content/data/dataset.txt) > /content/data/train.txt

In [6]:
# Sub-set de validacion
VAL_SIZE = 300 #@param {type:"integer"}
!(sed -n {TRAIN_SIZE + 1},{TRAIN_SIZE + VAL_SIZE}p /content/data/dataset.txt) > /content/data/dev.txt

##Tokenizador
Con RoBERTa, se utiliza un tokenizador BPE (*Byte-Pair Encoding*), el cual trabaja a nivel de bytes. Por medio de esto, su reperesentación del texto puede tomarse como un vocabulario "universal".

In [7]:
from tokenizers import ByteLevelBPETokenizer

path = "/content/data/train.txt"

# Se inicializa el tokenizador.
tokenizer = ByteLevelBPETokenizer()

# Se personaliza para el entrenamiento.
tokenizer.train(files=path,
                vocab_size=50265,
                min_frequency=2,
                special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])

# Se crea la carpeta "models" para guardarlo.
!mkdir -p "models/roberta"
tokenizer.save("models/roberta")

['models/roberta/vocab.json', 'models/roberta/merges.txt']

##Arquitectura del modelo
Debido a que RoBERTa utiliza la misma arquitectura que BERT, con la diferencia del tokenizador BPE y la implementación del enmascaramiento de palabras; se debe eliminar la configuración de BERT respecto al tokenizador y a la predicción de la siguiente oración.

In [8]:
import json
config = {
	"architectures": [
		"RobertaForMaskedLM"
	],
	"attention_probs_dropout_prob": 0.1,
	"hidden_act": "gelu",
	"hidden_dropout_prob": 0.1,
	"hidden_size": 768,
	"initializer_range": 0.02,
	"intermediate_size": 3072,
	"layer_norm_eps": 1e-05,
	"max_position_embeddings": 514,
	"model_type": "roberta",
	"num_attention_heads": 12,
	"num_hidden_layers": 12,
	"type_vocab_size": 1,
	"vocab_size": 50265
}

#Se eliminan las configuraciones por defecto de BERT.

with open("models/roberta/config.json", 'w') as fp:
    json.dump(config, fp)

tokenizer_config = {"max_len": 512}

with open("models/roberta/tokenizer_config.json", 'w') as fp:
    json.dump(tokenizer_config, fp)

##Preparar el entrenamiento
El modelo será entrenado utilizando *run_language_modeling.py*, un script proporcionado por *Hugging Face*, que preprocesa, tokeniza el corpus y entrena el modelo en *Masked Language Modeling (MLM).*

Se establecen, además, las direcciones del entrenamiento y evaluación del modelo.

In [9]:
!wget -c https://raw.githubusercontent.com/chriskhanhtran/spanish-bert/master/run_language_modeling.py

# Direcciones del modelo
MODEL_TYPE = "roberta" #@param ["roberta", "bert"]
MODEL_DIR = "models/roberta" #@param {type: "string"}
OUTPUT_DIR = "models/roberta" #@param {type: "string"}
#OUTPUT_DIR = "models/roberta/output" #@param {type: "string"}
TRAIN_PATH = "/content/data/train.txt" #@param {type: "string"}
EVAL_PATH = "/content/data/dev.txt" #@param {type: "string"}
TOKENIZER = "models/roberta"

--2021-12-08 22:00:02--  https://raw.githubusercontent.com/chriskhanhtran/spanish-bert/master/run_language_modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34328 (34K) [text/plain]
Saving to: ‘run_language_modeling.py’

run_language_modeli 100%[===================>]  33.52K  --.-KB/s    in 0.004s  

2021-12-08 22:00:03 (8.24 MB/s) - ‘run_language_modeling.py’ saved [34328/34328]



Se agregan a la línea de comandos las direcciones del modelo.

In [11]:
# Línea de comandos
cmd = """python run_language_modeling.py \
    --output_dir {output_dir} \
    --model_type {model_type} \
    --mlm \
    --config_name {config_name} \
    --tokenizer_name {tokenizer_name} \
    {line_by_line} \
    {should_continue} \
    {model_name_or_path} \
    --train_data_file {train_path} \
    --eval_data_file {eval_path} \
    --do_train \
    {do_eval} \
    {evaluate_during_training} \
    --overwrite_output_dir \
    --num_train_epochs 50 \
    --block_size 512 \
    --max_step 25 \
    --warmup_steps 50 \
    --learning_rate 5e-5 \
    --per_gpu_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --weight_decay 0.01 \
    --adam_epsilon 1e-6 \
    --max_grad_norm 100.0 \
    --save_total_limit 10 \
    --save_steps 10 \
    --logging_steps 2 \
    --seed 42
"""

Argumentos para iniciar el entrenamiento desde cero.

In [12]:
# Se descartaron las opciones de evaluate_during_training, line_by_line, should_continue, and model_name_or_path.
train_params = {
    "output_dir": OUTPUT_DIR,
    "model_type": MODEL_TYPE,
    "config_name": MODEL_DIR,
    "tokenizer_name": MODEL_DIR,
    "train_path": TRAIN_PATH,
    "eval_path": EVAL_PATH,
    "do_eval": "--do_eval",
    "evaluate_during_training": "",
    "line_by_line": "",
    "should_continue": "",
    "model_name_or_path": "",
}

##Entrenamiento
Este proceso puede tardar varios minutos.

In [13]:
!{cmd.format(**train_params)}

12/08/2021 22:00:54 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
12/08/2021 22:00:54 - INFO - transformers.configuration_utils -   loading configuration file models/roberta/config.json
12/08/2021 22:00:54 - INFO - transformers.configuration_utils -   Model config RobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "

##Predecir las palabras enmascaradas
El modelo aprende a rellenar las palabras enmascaradas al darle un contexto.

In [14]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="/content/models/roberta",
    tokenizer="/content/models/roberta"
)

Creating an empty model card.


##Introduzca la oración enmascarada.
¡Pruebe el modelo!

In [15]:
fill_mask("Debi Nova es <mask>.")

[{'score': 0.015443866141140461,
  'sequence': '<s> Debi Nova es de.</s>',
  'token': 269},
 {'score': 0.012500587850809097,
  'sequence': '<s> Debi Nova es que.</s>',
  'token': 281},
 {'score': 0.005881866440176964,
  'sequence': '<s> Debi Nova es y.</s>',
  'token': 286},
 {'score': 0.004724670667201281,
  'sequence': '<s> Debi Nova es la.</s>',
  'token': 290},
 {'score': 0.0024852119386196136,
  'sequence': '<s> Debi Nova es a.</s>',
  'token': 270}]

##Créditos
Tran, C. (2020, 31 diciembre). *Tutorial: How to train a RoBERTa Language Model for Spanish*. Skim AI. https://skimai.com/roberta-language-model-for-spanish/